# Импортируем библиотеки и ключ openai из файла token

In [1]:
import os
import langchain
from pprint import pprint as pp
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import warnings

token_file = open("../utils/token.txt", "r")
token = token_file.readline()
token_file.close()

os.environ["OPENAI_API_KEY"] = token

number_of_goods = 6

# Деление текста на фрагменты

In [2]:
data_plus = []
path = '../input/'
text_loader_kwargs={'autodetect_encoding': True}
splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    separators=["\n\n", "\n", "(?<=\. )"]
)
for i in range(number_of_goods):
    loader = DirectoryLoader(path, glob="text"+str(i)+".txt", loader_cls=TextLoader,
                             loader_kwargs=text_loader_kwargs)
    docs = loader.load()
    
    data_plus.append(splitter.split_documents(docs))
    
data_plus #  check

# splitter = RecursiveCharacterTextSplitter(
#     chunk_size=100,
#     chunk_overlap=0,
#     separators = ['. ', "\n"]
# )
# splitted_data = splitter.split_text(coffee_zero)
# splitted_data

# loader = DirectoryLoader(path, glob="plus1.txt", loader_cls=TextLoader,
#                          loader_kwargs=text_loader_kwargs)
# docs = loader.load()

# splitted_texts = splitter.split_documents(docs)
# splitted_texts

[[Document(page_content='отличная кофемашина, бюджетная для семьи 2 человека, простая в использовании.', metadata={'source': '../input/text0.txt'}),
  Document(page_content='1. Весьма удобно, после рожка  исчезли россыпи молотого кофе по всей кухне.', metadata={'source': '../input/text0.txt'}),
  Document(page_content='\n2. Имеется нормальный капучинатор, но я не пользовался еще, судя по отзывам качество посредственное, но его хотя бы не надо так тщательно вымывать после каждого использования, баклашку с молоком отцепили и поставили в холодильник, пениться будет лучше.', metadata={'source': '../input/text0.txt'}),
  Document(page_content='\n3. Несмотря на черный цвет  материалы не маркие и раз в неделю, можно просто стереть пыль даже сухой тряпкой.', metadata={'source': '../input/text0.txt'}),
  Document(page_content='\n4. Вместимости под зерна кофе хватает на дня 34, а лоток с водой и жмыхом примерно раз в сутки чищу.', metadata={'source': '../input/text0.txt'}),
  Document(page_conte

# Переводим данные в чисолвые векторы, сохраняем в БД 

In [4]:
directory = ''
embedding = OpenAIEmbeddings()

# vectordb = Chroma.from_documents(
#     documents=splitted_texts,
#     embedding=embedding,
#     persist_directory=directory
# )

# print(vectordb._collection.count()) # Проверка
# vectordb.persist() # Пишем
vectordb_list = []

for i in range(number_of_goods):
    vectordb_list.append(Chroma.from_documents(documents=data_plus[i], 
                                               embedding=embedding, persist_directory="../output/"+str(i)))
for j in range(number_of_goods):
    vectordb_list[j].persist()
    print(vectordb_list[j]._collection.count()) # Проверка
    
# # Читаем
# vectordb = Chroma(
#     persist_directory=directory,
#     embedding_function=embedding
# )
# print(vectordb._collection.count()) # Проверка
# # DEPRECATED

218
67
41
187
27
68


# Пишем ответ с помощью GPT

In [5]:
warnings.filterwarnings("ignore")
langchain.debug = False

question = "Расскажи про эту кофемашину"
docs_k = 65

template = """Ты - полезный ИИ консультант для нашего магазина бытовой техники по продаже кофемашин.
Твое задание - описать данную кофемашину. Говори только о достоинствах.
Используйте следующие фрагменты контекста (Context), чтобы ответить на вопрос в конце (Question).
Если вы не знаете ответа, просто скажите, что не знаете, не пытайтесь придумывать ответ.
Сначала убедитесь, что прикрепленный текст имеет отношение к вопросу.
Если вопрос не имеет отшения к тексту, ответьте, что вы не можете ответить на данный вопрос.
Используйте максимум 15 предложений. 
Дайте ответ как можно более понятным, рассказывая кратко про все достинства именно данной кофемашины.
Context: {context}
Question: {question}""" 
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

for vectordb in vectordb_list:
    retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": docs_k})
    llm = ChatOpenAI(
        model_name="gpt-3.5-turbo",
        temperature=0,
        max_tokens = 250)
    qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
        #chain_type="refine",
        #chain_type="map_reduce",
    )
    result = qa_chain({"query": question})
    print(vectordb_list.index(vectordb), "----------------------------------------------------------------")
    pp(result)



0 ----------------------------------------------------------------
{'query': 'Расскажи про эту кофемашину',
 'result': 'Эта кофемашина отличается простотой в использовании и '
           'приготовлении вкусного кофе. Она имеет встроенную кофемолку, что '
           'делает процесс приготовления кофе еще более удобным. Машина '
           'надежная и легкая в освоении, что делает ее идеальным выбором для '
           'домашнего использования. Кроме того, она имеет автоматический '
           'капучинатор, что позволяет приготовить вкусный капучино. В общем, '
           'это отличная кофемашина за свои деньги, которая радует своим '
           'качеством и удобством использования.',
 'source_documents': [Document(page_content='Отличная кофемашина.\nВарит вкусный кофе.', metadata={'source': '../input/text0.txt'}),
                      Document(page_content='Отличная кофемашина.\n"Варит вкусный кофе.', metadata={'source': '../input/plus0.txt'}),
                      Document(page_conten

Number of requested results 65 is greater than number of elements in index 41, updating n_results = 41


2 ----------------------------------------------------------------
{'query': 'Расскажи про эту кофемашину',
 'result': 'Эта кофемашина имеет множество достоинств. Она оснащена удобным '
           'сенсорным дисплеем, большим выбором напитков в базе и '
           'автоматическим капучинатором. Благодаря капучинатору кофе '
           'получается с воздушной, сливочной пенкой, как в кофейне. Также '
           'можно настроить напитки под свой вкус и регулировать их крепость. '
           'Кофемашина готовит вкусный кофе и другие напитки автоматически, '
           'что делает ее отличным выбором для любителей кофе.',
 'source_documents': [Document(page_content='\nОтличная современная кофемашина! Достоинства: удобный сенсорный дисплей, Много напитков в базе, автоматический капучинатор. Кофе получается как в кофейне за счет капучинатора - делает сливочную, воздушную пенку.', metadata={'source': '../input/plus2.txt'}),
                      Document(page_content='\nОтличная современная к

Number of requested results 65 is greater than number of elements in index 27, updating n_results = 27


4 ----------------------------------------------------------------
{'query': 'Расскажи про эту кофемашину',
 'result': 'Эта кофемашина отлично готовит кофе, супер нравится и варит его '
           'лучше, чем бариста! У нее есть автоматический капучинатор, большой '
           'выбор настроек крепости и объема кофе, и простота в обслуживании - '
           'легко моется стакан капучинатора. Она также хорошо работает даже '
           'на плохой воде благодаря встроенному фильтру. После использования '
           'рожка, россыпи молотого кофе по кухне больше не будет. '
           'Единственный недостаток - дороговата и немного шумновата, но это '
           'компенсируется легкостью использования и обслуживания.',
 'source_documents': [Document(page_content='\nВкусный кофе. Старая машина была более шумная. Шикарная пенка. Все настроила под себя с легкостью. Места занимает немного на моей кухне.', metadata={'source': '../input/plus4.txt'}),
                      Document(page_content='\